In [1]:
%matplotlib inline  
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import math

import pandas as pd

#Ploting
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster

import sys
sys.path.append('../scripts/')

from speed_matrix import SpeedMatrix

In [2]:
client = MongoClient()
geo4cast = client.geo4cast
data = geo4cast.coyote2

osm_rennes = client["osm-rennes"]
roads = osm_rennes.roads

In [4]:
def print_road(road_id):

    coords = roads.find({'_id': road_id})[0]['loc']['coordinates']

    folium_map = folium.Map(location=[48.11,-1.66],
                         zoom_start=13,
                         tiles="OpenStreetMap")
    folium.PolyLine(
        locations=[[x[1], x[0]] for x in coords],
        color='red'
    ).add_to(folium_map)

    return folium_map

In [10]:
sm = SpeedMatrix("geo4cast", "coyote2")
speeds = sm.get_speed_matrix_headings(15)
speeds.head(10)

time,2018-10-12 08:45:00,2018-10-12 09:00:00,2018-10-12 09:15:00,2018-10-12 09:30:00,2018-10-12 09:45:00,2018-10-12 10:00:00,2018-10-12 10:15:00,2018-10-12 10:30:00,2018-10-12 10:45:00,2018-10-12 11:00:00,...,2018-10-14 14:45:00,2018-10-14 15:00:00,2018-10-14 15:15:00,2018-10-14 15:30:00,2018-10-14 15:45:00,2018-10-14 16:00:00,2018-10-14 16:15:00,2018-10-14 16:30:00,2018-10-14 16:45:00,2018-10-14 17:00:00
matching_road,,,,,,,,,,,,,,,,,,,,,
100319558_0,11.000000,NaN,NaN,25.0,NaN,NaN,0.0,NaN,NaN,0.0,...,11.0,8.5,NaN,NaN,NaN,NaN,NaN,NaN,37.0,0.0
100319558_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
100322963_0,45.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322963_1,9.166667,NaN,23.666667,54.0,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322964_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322964_1,NaN,NaN,5.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057060_0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057061_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057061_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
abbr = speeds.loc['25620793_0'].dropna()
abbr

time
2018-10-12 08:45:00     39.400000
2018-10-12 09:00:00     38.333333
2018-10-12 12:45:00     39.000000
2018-10-12 13:15:00     66.000000
2018-10-12 14:45:00     40.000000
2018-10-12 15:45:00     70.500000
2018-10-12 16:00:00     55.000000
2018-10-12 16:15:00     46.000000
2018-10-12 16:30:00     43.000000
2018-10-12 17:00:00     46.000000
2018-10-12 18:00:00     49.500000
2018-10-12 18:15:00     40.666667
2018-10-12 18:30:00     41.500000
2018-10-12 19:00:00     15.000000
2018-10-12 19:15:00      0.000000
2018-10-12 19:30:00      4.133333
2018-10-12 19:45:00      0.000000
2018-10-12 20:00:00     39.333333
2018-10-12 20:15:00     45.000000
2018-10-12 23:00:00     66.000000
2018-10-12 23:15:00     55.000000
2018-10-12 23:45:00      0.000000
2018-10-13 00:15:00      0.000000
2018-10-13 00:30:00      0.000000
2018-10-13 02:15:00     34.500000
2018-10-13 09:00:00     38.333333
2018-10-13 13:15:00    130.000000
2018-10-13 14:00:00      0.000000
2018-10-13 14:15:00      0.000000
2018-10-1

In [32]:
groupped = list(data.aggregate([
    { "$match": { "speed": 0 } },
    { "$group": {
        "_id": {
            "matching_road": "$matching_road",
            "id": "$id",
            "time": {
                "$toDate": {
                    "$subtract": [
                        { "$toLong": "$time"},
                        { "$mod": [ { "$toLong": "$time"}, 1000 * 60 * 15] }
                        ]
                    }
                }
            },
        "ids": { "$push": "$_id"},
        "count": {"$sum": 1} 
        }},
    { "$match": { "count": { "$gt": 5 } } }
]))

In [36]:
groupped[0]['ids'][1:]

[ObjectId('5bc08460e7d57702cc045d32'),
 ObjectId('5bc08492e7d57702cc065f16'),
 ObjectId('5bc084b4e7d57702cc07b27e'),
 ObjectId('5bc084a4e7d57702cc070615'),
 ObjectId('5bc08470e7d57702cc05081a'),
 ObjectId('5bc084d4e7d57702cc0905d0'),
 ObjectId('5bc084e6e7d57702cc09aaa1'),
 ObjectId('5bc08483e7d57702cc05b397'),
 ObjectId('5bc084f6e7d57702cc0a561f'),
 ObjectId('5bc084c5e7d57702cc085fb7')]

In [40]:
ids = []
for g in groupped:
    ids += g['ids'][1:]
ids 

[ObjectId('5bc08460e7d57702cc045d32'),
 ObjectId('5bc08492e7d57702cc065f16'),
 ObjectId('5bc084b4e7d57702cc07b27e'),
 ObjectId('5bc084a4e7d57702cc070615'),
 ObjectId('5bc08470e7d57702cc05081a'),
 ObjectId('5bc084d4e7d57702cc0905d0'),
 ObjectId('5bc084e6e7d57702cc09aaa1'),
 ObjectId('5bc08483e7d57702cc05b397'),
 ObjectId('5bc084f6e7d57702cc0a561f'),
 ObjectId('5bc084c5e7d57702cc085fb7'),
 ObjectId('5bc06eb5e7d5777f4b478c90'),
 ObjectId('5bc06ec9e7d5777f4b485429'),
 ObjectId('5bc06e8ee7d5777f4b45f9e8'),
 ObjectId('5bc06e7ae7d5777f4b4534ac'),
 ObjectId('5bc06ea0e7d5777f4b46bd0e'),
 ObjectId('5bc06edce7d5777f4b491a2f'),
 ObjectId('5bc06ef2e7d5777f4b49e3bf'),
 ObjectId('5bc06f06e7d5777f4b4ab426'),
 ObjectId('5bc06f19e7d5777f4b4b7f00'),
 ObjectId('5bc1773ae7d5773942c0d590'),
 ObjectId('5bc17799e7d5773942c1b057'),
 ObjectId('5bc177fee7d5773942c28358'),
 ObjectId('5bc1793ee7d5773942c50b6c'),
 ObjectId('5bc179a1e7d5773942c5e69c'),
 ObjectId('5bc178d6e7d5773942c4375a'),
 ObjectId('5bc179fde7d577

In [43]:
res = data.delete_many({'_id': { "$in": ids}})
res.deleted_count

In [44]:
sm = SpeedMatrix("geo4cast", "coyote2")
speeds = sm.get_speed_matrix_headings(15)
speeds.head(10)

time,2018-10-12 08:45:00,2018-10-12 09:00:00,2018-10-12 09:15:00,2018-10-12 09:30:00,2018-10-12 09:45:00,2018-10-12 10:00:00,2018-10-12 10:15:00,2018-10-12 10:30:00,2018-10-12 10:45:00,2018-10-12 11:00:00,...,2018-10-14 14:45:00,2018-10-14 15:00:00,2018-10-14 15:15:00,2018-10-14 15:30:00,2018-10-14 15:45:00,2018-10-14 16:00:00,2018-10-14 16:15:00,2018-10-14 16:30:00,2018-10-14 16:45:00,2018-10-14 17:00:00
matching_road,,,,,,,,,,,,,,,,,,,,,
100319558_0,11.000000,NaN,NaN,25.0,NaN,NaN,0.0,NaN,NaN,0.0,...,11.0,8.5,NaN,NaN,NaN,NaN,NaN,NaN,37.0,0.0
100319558_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
100322963_0,45.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322963_1,9.166667,NaN,23.666667,54.0,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322964_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100322964_1,NaN,NaN,5.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057060_0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057061_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10057061_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
abbr = speeds.loc['25620793_0'].dropna()
abbr

time
2018-10-12 08:45:00     39.400000
2018-10-12 09:00:00     38.333333
2018-10-12 12:45:00     39.000000
2018-10-12 13:15:00     66.000000
2018-10-12 14:45:00     40.000000
2018-10-12 15:45:00     70.500000
2018-10-12 16:00:00     55.000000
2018-10-12 16:15:00     46.000000
2018-10-12 16:30:00     43.000000
2018-10-12 17:00:00     46.000000
2018-10-12 18:00:00     49.500000
2018-10-12 18:15:00     40.666667
2018-10-12 18:30:00     41.500000
2018-10-12 19:00:00     33.000000
2018-10-12 19:15:00      0.000000
2018-10-12 19:30:00     31.000000
2018-10-12 19:45:00      0.000000
2018-10-12 20:00:00     39.333333
2018-10-12 20:15:00     45.000000
2018-10-12 23:00:00     66.000000
2018-10-12 23:15:00     55.000000
2018-10-12 23:45:00      0.000000
2018-10-13 00:15:00      0.000000
2018-10-13 00:30:00      0.000000
2018-10-13 02:15:00     34.500000
2018-10-13 09:00:00     38.333333
2018-10-13 13:15:00    130.000000
2018-10-13 14:00:00      0.000000
2018-10-13 14:15:00      0.000000
2018-10-1

In [3]:
from cleaner import remove_aberrant

In [5]:
remove_aberrant(data, 4)

/home/maxime/.local/lib/python3.6/site-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/home/maxime/.local/lib/python3.6/site-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/home/maxime/.local/lib/python3.6/site-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/home/maxime/.local

21809 points deleted
21807 points deleted
21809 points deleted
21809 points deleted
[21809, 21809, 21809, 21807]
87234 points deleted
